In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

# Visualizing the processed data
In this notebook, we aim to visualize our core components of the project.
## 1. Timeline visualization
We'll start with creating a timeline for each name, and display a certain amount of movies (for example the top 10) that came out throughout the years.

In [ ]:
folder_processed_data_path = './data/processed_data/'

movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
movie_df.set_index(['wiki_ID'], inplace=True)

name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_df.csv'))
name_by_movie_df.set_index(['wiki_ID', 'char_words', 'gender'], inplace=True)

baby_name_df = pd.read_csv(os.path.join(folder_processed_data_path, 'baby_name_df.csv'))
baby_name_df.set_index(['name', 'year'], inplace=True)

**Visualisation for a specific name:** Let's choose a name and visualize the variation of the baby names throughout the years. We'll pick `Mia`, because it's a cool name.

In [ ]:
# Set a name
chosen_name = "Mia"

# Filter the name_by_movie_df by the chosen name
filt_name_by_movie_df = name_by_movie_df.query("char_words == @chosen_name").copy(deep=True)
filt_name_by_movie_df.reset_index(inplace=True)

display(filt_name_by_movie_df.sort_values(by="wiki_ID").head())
print(f"There are {len(filt_name_by_movie_df)} movies with a character named {chosen_name}.")

In [ ]:
# Keep only wiki_ID column
filt_name_by_movie_df = filt_name_by_movie_df[['wiki_ID']].copy(deep=True)

# Merge the movie_df with the filt_name_by_movie_df
filt_movie_df = pd.merge(movie_df, filt_name_by_movie_df, on='wiki_ID', how='inner').copy(deep=True)
display(filt_movie_df.sort_values(by="wiki_ID").head())

There are too many movies. Let's keep only the 10 most famous ones for now. We base it on the `numVotes` column, representing the number of IMDB votes:

In [ ]:
# We set the number of top movies for this visualization
top_movie_count = 10

# Sort by top rated movies
sorted_filt_movie_df = filt_movie_df.sort_values(by='numVotes', ascending=False).copy(deep=True)

# Keep only the top 10 movies
filt_movie_df = sorted_filt_movie_df.head(top_movie_count).copy(deep=True)
display(filt_movie_df)

# Get the release dates
release_serie = filt_movie_df['release']

We now plot the percentage of babies having the chosen name `Mia` for each year:

In [ ]:
# Grab percentage data
percentage_df = baby_name_df.loc[chosen_name]['percentage'].to_frame().reset_index()
percentage_df.sort_values(by=['year'], ascending=True, inplace=True)
print(f"There are {len(percentage_df)} years with a character named {chosen_name}. \nThe earliest year is {percentage_df['year'].min()} and the latest is {percentage_df['year'].max()}.")

x_values = percentage_df['year'].values
y_values = percentage_df['percentage'].values

# Plotting
plt.plot(x_values, y_values)
plt.xlabel('Years')
plt.ylabel('Percentage of total births')
plt.title(f'Name "{chosen_name}" with ten most reviewed \n movies with a character of the same name')

# Draw vertical lines for each movie release
vertical_lines_series = release_serie.values
for x_value in vertical_lines_series:
    plt.axvline(x=x_value, color='r', linestyle='--')

# Labeling
plt.axvline(x=x_value, color='r', linestyle='--', label='Movie releases')
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
plt.show()

We note that there are not 10 marks on the graphs, that's because some movies had the same release year. We won't be able to distinguish them in our analysis and they will be considered as having the same impact (which would be in reality their "cumulated" impact).

## 2. Parallel questions 
### a. Distribution of months in movies
Looking at studies showing that baby conception rates are at the highest in fall or winter season leading to higher birth in the summer, will movies released in summer show the highest correlation with newborn naming?

In [ ]:
# How many movies have months?

# Plot the distribution of months for movies

### b. Movie genres
Does the genre of a movie play a role in the influence of character names on baby naming trends?

In [ ]:
# Import the genres
movie_genres_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_genres_df.csv'))

In [ ]:
# Plot the number of movies per genre
plt.figure(figsize=(10, 12))
sns.countplot(y='genre', data=movie_genres_df, order=movie_genres_df['genre'].value_counts().index[:40]) # only take top 40 genres

plt.title('Number of Movies per Genre')
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.show()

# Count the number of genres in movies
genre_counts = movie_genres_df.groupby('wiki_ID')['genre'].count()
genre_count_frequency = genre_counts.value_counts().sort_index()

genre_count_frequency.plot(kind='bar')
plt.title('Number of Genres per Movie')
plt.xlabel('Number of Genres')
plt.ylabel('Number of Movies')
plt.yscale('log')
plt.show()

In [ ]:
top_5_genres = movie_genres_df['genre'].value_counts().index[:5].values
print(top_5_genres)

We can see that the most popular genres are these one above.

### c. Movie popularity
We compute the movie's popularity based on their number of votes on IMDB, which we now have.

In [ ]:
# Plot number of movies per year 
movie_df

In [ ]:
# Show the total number of ratings per year
movie_df

In [ ]:
# Now we want to normalize the number of ratings by the number of movies released in that year 
# to see if the average number of ratings per movie has changed over time

In [ ]:
# Which movie genres are the most popular? (sum of ratings)

### d. Importance of characters in movies
Importance is based on the TMDB dataset, represented by orders. The lower the number, the more important the character.

In [ ]:
# Plot the histogram of number of characters for movies
name_by_movie_df

# Plot a boxplot of the mean number of characters per movie (cause why not)


In [ ]:
# How many movies have ordered characters? (all characters in the movie have NaN values for orders)

### e. Genders of movie characters
We can check how the character genders are distributed in the movies. It's important to remember that this doesn't show the number of male or female characters, but rather the number of names derived from the character names. For example a female character that has 3 names will contribute with 3 entries in our dataset.

In [ ]:
# Count the number of genders in all movies, keep nan values
names_by_gender = name_by_movie_df.groupby('gender', dropna=False).count()
names_by_gender = names_by_gender.rename(columns={'order': 'genders_count'})
display(names_by_gender)

# Create pie chart of the counts
plt.figure(figsize=(5, 5))
plt.pie(names_by_gender.genders_count, labels=names_by_gender.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('twilight_shifted', 3))

plt.show()

Let's check the ratio of M/F/NaN for some movie genres, for exmple the top 10:

In [ ]:
# Grab top 10 genres and order them by count
top_genres = movie_genres_df['genre'].value_counts().nlargest(10)
display(top_genres)

# Merge on the genre column to keep only the top 10 genre movies
top_genre_movies_ids_df = pd.merge(movie_genres_df, top_genres, on='genre', how='inner').drop(columns=['count'])
display(top_genre_movies_ids_df.head())

# Merge to keep the character names with the genres only
names_per_genre = pd.merge(top_genre_movies_ids_df, name_by_movie_df, on='wiki_ID', how='left')
display(names_per_genre.head())

# Group by genre, count different genders
gender_counts_per_genre = names_per_genre.groupby(['genre', 'gender'], dropna=False).count()
gender_counts_per_genre = gender_counts_per_genre.rename(columns={'wiki_ID': 'count'}).drop(columns=['index', 'char_words', 'order'])
display(gender_counts_per_genre.head(9))

In [ ]:
# Pl